In [ ]:
import pandas as pd
import decimal 

-Format EIA Data-

In [ ]:
# The EIA data table is formatted with notes at the beginning and end, drop them from dataframe
# see skiprows and skipfooter
eiaDropHead = [0, 1]

# Columns we don't need for GGPT (will drop later, putting here for visibility)
eiaDropCols = ['Sector', 'Net Summer Capacity (MW)', 'Net Winter Capacity (MW)', 'Technology', 'Operating Month', 'Planned Retirement Month', 'Retirement Month', 'Planned Operation Month',
'Google Map', 'Bing Map', 'Balancing Authority Code', 'Planned Derate Year', 'Planned Derate Month', 'Planned Derate of Summer Capacity (MW)', 'Planned Uprate Year', 'Planned Uprate Month', 
'Planned Uprate of Summer Capacity (MW)']

# EIA list of tabs to merge
eiaTabs = ['Operating', 'Planned', 'Retired', 'Canceled or Postponed', 'Operating_PR', 'Planned_PR', 'Retired_PR']

eiaList = [] # for storing each tab to merge later, better performance and this is a large table)
eiaLenQC = 0

# Loop over each sheet to read in and merge
for i in eiaTabs:
    print("Processing tab: ", i)

    eia_ = pd.read_excel(r'C:\Users\Sarah\Documents\GEM\research\North America\US\EIA\july_generator2021.xlsx', sheet_name=i, skiprows=eiaDropHead, skipfooter=2, keep_default_na=False)
    eiaLenQC += len(eia_.index) # Can compare this number later to the sum of all rows in the EIA Excel sheet

    # Keep only gas-fired plants
    eia_ = eia_.loc[eia_['Energy Source Code'] == 'NG']
   
    # Add status column based on the tab's name, EIA Status column inconsistent
    eia_['Status_Tab'] = i.lower()

    # Store this tab's data to merge
    eiaList += [eia_]

# Combine tabs
eia = pd.concat(eiaList)

# Drop columns
eia.drop(columns=eiaDropCols, inplace=True)

# Total rows
print('\n')
print("Total row count of all tabs: ", eiaLenQC, '\n')
print("Total row count, only NG-fired units: ", len(eia.index), '\n')


Merge combined cycle units into single sets

In [ ]:
# Merge units into proper sets
# 'ST', 'GT', and 'CS' are individual. 'CA' (steam) and 'CT' (gas turbine) need to be combined to 'CC'
ccSets = eia.loc[eia['Prime Mover Code'].isin(['CA', 'CT'])]
ccToRemove = ccSets.index

# Group by plant ID and Unit Code fields
# Sum capacities, group Generator IDs, keep sets of values that could possibly vary between units
ccGroup = ccSets.groupby(['Plant ID', 'Unit Code', 'Status_Tab'], as_index=False).agg({'Nameplate Capacity (MW)': 'sum', 'Generator ID': lambda x: list(x), 'Status': lambda x: set(x),
'Operating Year': lambda x: set(x), 'Planned Retirement Year': lambda x: set(x), 'Planned Operation Year': lambda x: set(x), 'Retirement Year': lambda x: set(x), 'Entity ID': lambda x: set(x),
'Entity Name': lambda x: set(x)})

# Set the correct tech code
ccGroup['Prime Mover Code'] = 'CC'



In [ ]:
# Check Sets for multiple values and turn into a range if necessary (e.g. different start years for units of a CC generator)
checkSets = ['Status', 'Operating Year', 'Planned Retirement Year', 'Planned Operation Year', 'Retirement Year', 'Entity ID','Entity Name']
for i in checkSets:
    # Wherever there are multiple values print them to screen
    print(i)
    print(len(ccGroup.loc[ccGroup[i].apply(len) != 1].index), '\n')
    # IF it's list of NaN, remove
    check = []
    check = ccGroup.loc[ccGroup[i].apply(len) != 1]
    if len(check) > 0 and 'Year' in i:
        ccGroup.loc[ccGroup[i].apply(len) != 1, i] = ccGroup.loc[ccGroup[i].apply(len) != 1][i].apply(lambda x: str(int(min(list(x)))) + '-' +  str(int(max(list(x)))) )
        # TODO: nan outputs

    # Wherever there's just one value, replace the set with the value
    # ccGroup.loc[ccGroup[i].apply(len) == 1, i] = ccGroup.loc[ccGroup[i].apply(len) == 1][i].apply(lambda x: list(x)[0])

In [ ]:
# Join back relevant plant-level columns
eiaBackfill = eia.drop_duplicates(subset='Plant ID', ignore_index=True)
eiaBackfill = eiaBackfill.drop(columns=['Unit Code', 'Nameplate Capacity (MW)', 'Generator ID', 'Status', 'Operating Year', 'Planned Retirement Year', 'Planned Operation Year',
'Retirement Year', 'Entity ID', 'Entity Name', 'Prime Mover Code', 'Status_Tab'])
ccGroupJoin = ccGroup.merge(eiaBackfill, on='Plant ID', how='left')
print(ccGroupJoin.columns)

In [ ]:
# Remove units from eia dataset, add back-in the new sets
eiaFormatted = eia.drop(index=ccToRemove)
eiaFormatted = eiaFormatted.append(ccGroupJoin, ignore_index=True)

In [ ]:
# remove units <50 MW
tmp = pd.to_numeric(eiaFormatted['Nameplate Capacity (MW)'], errors='coerce')
print("Units missing capacity data or other errors: ", '\n', eiaFormatted.loc[tmp.isna()])
eiaFormatted.drop(index=eiaFormatted.loc[tmp.isna()].index, inplace=True)
eiaFormatted['Nameplate Capacity (MW)'] = eiaFormatted['Nameplate Capacity (MW)'].apply(lambda x: decimal.Decimal(x).quantize(decimal.Decimal('1'), 
    rounding=decimal.ROUND_HALF_UP)) # Python 3 went to Round Half Even, overriding
eiaFormatted.drop(index=eiaFormatted.loc[eiaFormatted['Nameplate Capacity (MW)'] < 50].index, inplace=True)

In [ ]:
# Change 'CS' to 'CC'
eiaFormatted.loc[eiaFormatted['Prime Mover Code'] == 'CS', 'Prime Mover Code'] = 'CC'

In [ ]:
# Create 'Other ID' fields to match GGPT

In [ ]:
eiaFormatted.to_excel(r'C:\Users\Sarah\Documents\GEM\code\GGPT-US-EIA\scratch\eiaFormatted.xlsx', index=False)

Compare new EIA data to existing GGPT values


In [ ]:
# Match IDs to GGPT and compare field values (status, capacity, year)

# Locate any new unit IDs

# Check capacity for any new units

In [ ]:
# Check for missing dual-fuel plants in EIA-860


In [ ]:
# Add in ownership information from separate EIA table